<a href="https://colab.research.google.com/github/ww-Jing/Algorithmic_Trading/blob/main/Simple_RSI_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
import numpy as np
import pandas as pd 
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## **獲取資料**

In [42]:
end = datetime.now()
start = datetime(end.year -10 , end.month, end.day)

df = yf.download('GOOG', start, end)

[*********************100%***********************]  1 of 1 completed


## **撰寫策略**

In [43]:
def rsi(values, period = 60) : 
  up = values[values > 0 ].mean()
  down = abs(values[values < 0 ]).mean()
  return (up/(up+down) )*100

In [44]:
df['Momentum_1D'] = (df['Close']-df['Close'].shift(1)).fillna(0)
df['RSI'] = df['Momentum_1D'].rolling(center=False, window=60).apply(rsi).fillna(0)
df['Volume_plain'] = df['Volume'].fillna(0)
df.tail(5)

,Open,High,Low,Close,Adj Close,Volume,Momentum_1D,RSI,Volume_plain
Date,,,,,,,,,
2023-03-31,101.709999,104.190002,101.440002,104.000000,104.000000,28086500,2.680000,53.381247,28086500
2023-04-03,102.669998,104.949997,102.379997,104.910004,104.910004,20719900,0.910004,53.051308,20719900
2023-04-04,104.839996,106.099998,104.599998,105.120003,105.120003,20377200,0.209999,52.599941,20377200
2023-04-05,106.120003,106.540001,104.101997,104.949997,104.949997,21864200,-0.170006,53.934422,21864200
2023-04-06,105.769997,109.629997,104.815002,108.900002,108.900002,34684200,3.950005,55.259491,34684200


In [49]:
#策略：： RSI14 < 40 買進，，RSI > 60 賣出
rsi = df['RSI'].tolist()
#訊號標籤
sig = []

stock = 0 

#偵測訊號
for i in range(len(rsi)):
  if rsi[i] > 60 and stock == 1 :
    stock -= 1 
    sig.append(-1)
  elif rsi[i] < 40 and stock == 0:
    stock += 1 
    sig.append(1)
  else: 
    sig.append(0)


In [50]:
price = df['Close'].to_numpy()

rets = []
stock = 0 
buy_price = 0
sell_price = 0

for j in range(len(sig)):
  if sig[j] == 1 : 
    buy_price = price[j+1]
    rets.append((sell_price - buy_price) / buy_price)
    #stock += 1
  elif sig[j] == -1 :
    sell_price = price[j+1]
    rets.append((sell_price - buy_price) / buy_price)
    #stock -= 1

#print(np.delete(rets,[0]))
print(buy_price)
print(sell_price)
print(rets)
rets = np.delete(rets,[0])
print(rets)

total_ret = 1
for k in range(len(rets)):
  total_ret *= 1 + rets[k]

print(str(round((total_ret-1)*100,4))+'%')


133.2655029296875
108.80000305175781
[-1.0, 0.2697106675355376, -0.007409795327729437, 0.31680236984957366, -0.007841176698252784, 0.16155397346528727, -0.16478001908990556, 0.10371653879177332, 0.021093768808903554, 0.257963696884226, 0.21250794072816495, 0.282013377757001, 0.043541396640561426, 0.4298689669275919, -0.31310432660291965, -0.18358464373813216]
[ 0.26971067 -0.0074098   0.31680237 -0.00784118  0.16155397 -0.16478002
  0.10371654  0.02109377  0.2579637   0.21250794  0.28201338  0.0435414
  0.42986897 -0.31310433 -0.18358464]
194.5731%
